In [1]:
import pandas as pd
from sklearn import datasets
from sklearn.linear_model import LinearRegression

# Import Statisfactory
from statisfactory import Artefact, Craft, Pipeline, Session, Volatile

2021-10-28 12:54:36,850 [statisfactory.IO.artefacts.backend] : DEBUG : Registering 's3' backend
2021-10-28 12:54:36,851 [statisfactory.IO.artefacts.backend] : DEBUG : Registering '' backend
2021-10-28 12:54:36,852 [statisfactory.IO.artefacts.backend] : DEBUG : Registering 'lakefs' backend
2021-10-28 12:54:36,853 [statisfactory.IO.artefacts.artefact_interactor] : DEBUG : registering 'csv' interactor
2021-10-28 12:54:36,853 [statisfactory.IO.artefacts.artefact_interactor] : DEBUG : registering 'xslx' interactor
2021-10-28 12:54:36,854 [statisfactory.IO.artefacts.artefact_interactor] : DEBUG : registering 'pickle' interactor
2021-10-28 12:54:36,854 [statisfactory.IO.artefacts.artefact_interactor] : DEBUG : registering 'odbc' interactor
2021-10-28 12:54:36,855 [statisfactory.IO.artefacts.artefact_interactor] : DEBUG : registering 'datapane' interactor
2021-10-28 12:54:36,855 [statisfactory.IO.artefacts.artefact_interactor] : DEBUG : registering 'binary' interactor
2021-10-28 12:54:36,902 [

In [2]:
import pandas as pd
from sklearn import datasets
from sklearn.linear_model import LinearRegression
import datapane as dp
# Import Statisfactory
from statisfactory import Artefact, Craft, Pipeline, Session, Volatile

sess = Session(root_folder="exemples/dummyRepo/")
# Load a context and get the catalog from it.
catalog = sess.catalog
pipelines = sess.pipelines_definitions
config = sess.pipelines_configurations

2021-10-28 12:54:39,828 [statisfactory.session.session] : INFO : Initiating Statisfactory to : 'exemples/dummyRepo'
2021-10-28 12:54:39,948 [statisfactory.session.session] : INFO : adding 'Lib' to PATH
2021-10-28 12:54:40,002 [statisfactory.operator.craft] : DEBUG : adding Craft 'craft_spam' into 'spamer'
2021-10-28 12:54:40,003 [statisfactory.operator.craft] : DEBUG : adding Craft 'craft_foo' into 'fooer'
2021-10-28 12:54:40,004 [statisfactory.operator.pipeline.pipeline] : DEBUG : merging pipeline 'fooer' into 'full'
2021-10-28 12:54:40,004 [statisfactory.operator.craft] : DEBUG : adding Craft 'craft_spam' into 'spamer'
2021-10-28 12:54:40,004 [statisfactory.operator.pipeline.pipeline] : DEBUG : merging pipeline 'spamer' into 'full'
2021-10-28 12:54:40,005 [statisfactory.operator.craft] : DEBUG : adding Craft 'craft_foo' into 'fooer'
2021-10-28 12:54:40,161 [statisfactory.session.session] : DEBUG : The LakeFS exemple repo already exists.
2021-10-28 12:54:40,162 [statisfactory.session.

In [4]:
@Craft()
def foo() -> Artefact("coeffs_minio_lake"):
    
    out = pd.DataFrame({"a":[1,2,3]})
    return out

p = foo
with sess:
    p(samples=10, ref=3)

2021-10-28 12:49:19,909 [statisfactory.IO.artefacts.artefact_interactor] : DEBUG : saving 'pickle' : coeffs_minio_lake
2021-10-28 12:49:20,043 [statisfactory.operator.craft] : DEBUG : craft 'foo' : capturing artefacts : 'coeffs_minio_lake'


In [14]:
from inspect import signature, Parameter
from typing import Callable, Dict, Any

def _dispatch(callable: Callable, **kwargs) -> Dict[str, Any]:
    """
    Dispatch the variadic arguments the callable.

    Args:
        callable (Callable): The callable to dispatch arguments to.
    """

    sng = signature(callable)

    # If the callable accepts variadic keywords args, send them all
    has_variadics = any(p for p in sng.parameters.values() if p.kind == Parameter.VAR_KEYWORD)
    if has_variadics:
        return kwargs
    
    # Filter out non used arguments
    valids = set(p for p in sng.parameters.values())
    args = {k:v for k, v in kwargs.items() if k in valids}

    return args


    




{'a': 1}

In [6]:
@Craft()
def foo() -> Artefact("coeffs_minio"):
    
    out = pd.DataFrame({"a":[1,2,3]})
    return out

@Craft()
def bar() -> Artefact("coeffs_minio_xlsx"):
    
    out = pd.DataFrame({"a":[1,2,3] * 10000})
    return out

@Craft()
def spam(coeffs_minio: Artefact) ->Artefact("report_lake"):
    r = dp.Report(dp.Table(coeffs_minio))
    return r

@Craft()
def spam2(coeffs_minio: Artefact) ->Artefact("report"):
    r = dp.Report(dp.Table(coeffs_minio))
    return r
    

In [9]:
p = foo + spam + bar + spam2
#p.plot()
with sess:
    p(samples=10)

2021-10-28 11:47:43,743 [statisfactory.operator.craft] : DEBUG : adding Craft 'foo' into 'noName'
2021-10-28 11:47:43,744 [statisfactory.operator.craft] : DEBUG : adding Craft 'spam' into 'noName'
2021-10-28 11:47:43,744 [statisfactory.operator.craft] : DEBUG : adding Craft 'bar' into 'noName'
2021-10-28 11:47:43,745 [statisfactory.operator.craft] : DEBUG : adding Craft 'spam2' into 'noName'
2021-10-28 11:47:43,745 [statisfactory.operator.pipeline.pipeline] : INFO : Starting pipeline 'noName' execution
2021-10-28 11:47:43,746 [statisfactory.operator.pipeline.runner] : INFO : running craft 'foo'.
2021-10-28 11:47:43,746 [statisfactory.operator.pipeline.runner] : INFO : Executing foo with execution context : 
 {'samples': 10}
2021-10-28 11:47:43,747 [statisfactory.IO.artefacts.artefact_interactor] : DEBUG : saving 'pickle' : coeffs_minio
2021-10-28 11:47:44,126 [statisfactory.operator.craft] : DEBUG : craft 'foo' : capturing artefacts : 'coeffs_minio'
2021-10-28 11:47:44,126 [statisfacto

In [3]:
catalog = sess.catalog
out = catalog.load("coeffs_minio_lake", samples=10, lake_ref="main")

2021-10-28 12:54:47,436 [statisfactory.IO.artefacts.artefact_interactor] : DEBUG : loading 'pickle' : coeffs_minio_lake
